## General workflow for the analysis of the predictions of the SyntaxSQL
### The spider evaluation scripts only prints out the wrong queries. 
### As the authors have made available the output of the evaluation script, which contains the wrongly predicted queries. 
### IT is possible to find how many/which queries are different from the our gold development set. Which comprises 4 databases.

## Open the dev results
### We changed the path twice to the augmented version of syntax sql and the normal baseline, and reruned the code to find out the % of correct queries.

In [2]:
import pandas as pd 
data = pd.read_csv("/Users/yuriy/Downloads/generated_datasets/generated_data_augment/saved_models/dev_result_eval.txt", sep="\t", header=None)
data


,0
0,"medium pred: select Song_Name,Country,Name fro..."
1,"medium gold: SELECT name , country , age FRO..."
2,"medium pred: select Song_Name,Name,Country fro..."
3,"medium gold: SELECT name , country , age FRO..."
4,"medium pred: select avg(Age),min(Age),max(Age)..."
...,...
1509,group 0.583 0.45...
1510,order 0.600 0.46...
1511,and/or 0.978 0.92...
1512,IUEN 1.000 1.00...


## Slice the data to the correct row, uncoment the cell for the augmented version 

In [3]:
filtered_data= data[:1477]# <---- For the augmented version SyntaxSQLAugmented
#filtered_data= data[:1621] # <------- For the normal version of SyntaxSQL

In [4]:
filtered_data

,0
0,"medium pred: select Song_Name,Country,Name fro..."
1,"medium gold: SELECT name , country , age FRO..."
2,"medium pred: select Song_Name,Name,Country fro..."
3,"medium gold: SELECT name , country , age FRO..."
4,"medium pred: select avg(Age),min(Age),max(Age)..."
...,...
1472,medium gold: SELECT T2.feature_type_name FROM ...
1473,medium pred: select property_type_description ...
1474,medium gold: SELECT T2.property_type_descripti...
1475,hard pred: select property_name from Propertie...


## Open the preselected databases of the development set


In [376]:
dev_gold = pd.read_csv("/Users/yuriy/Desktop/Neural-Rule-Based-model-for-Text-to-SQL/data/devPreSelected.csv", index_col=0)
dev_gold

,database,question,sql
0,poker_player,How many poker players are there?,SELECT count(*) FROM poker_player
1,poker_player,Count the number of poker players.,SELECT count(*) FROM poker_player
2,poker_player,List the earnings of poker players in descending order.,SELECT Earnings FROM poker_player ORDER BY Earnings DESC
3,poker_player,"What are the earnings of poker players, ordered descending by value?",SELECT Earnings FROM poker_player ORDER BY Earnings DESC
4,poker_player,List the final tables made and the best finishes of poker players.,"SELECT Final_Table_Made , Best_Finish FROM poker_player"
...,...,...,...
115,singer,What are the names of the singers that have more than one songs?,SELECT T1.Name FROM singer AS T1 JOIN song AS T2 ON T1.Singer_ID = T2.Singer_ID GROUP BY T1.Name HAVING COUNT(*) > 1
116,singer,Show the names of singers and the total sales of their songs.,"SELECT T1.Name , sum(T2.Sales) FROM singer AS T1 JOIN song AS T2 ON T1.Singer_ID = T2.Singer_ID GROUP BY T1.Name"
117,singer,"For each singer name, what is the total sales for their songs?","SELECT T1.Name , sum(T2.Sales) FROM singer AS T1 JOIN song AS T2 ON T1.Singer_ID = T2.Singer_ID GROUP BY T1.Name"
118,singer,List the name of singers that do not have any song.,SELECT Name FROM singer WHERE Singer_ID NOT IN (SELECT Singer_ID FROM song)


## Filter the data from the wrong predicitions so there is only gold values, as we can compare them to the gold values of our development set and then count how many are missing (which would be the correct ones, as the evaluation script of spider only show the wrong queries)

In [377]:
filtered_data = filtered_data[0].str.split(":", n=1, expand=True)
filtered_data.columns = ['Column1', 'Column2']
filtered_data

,Column1,Column2
0,medium pred,"select Song_Name,Country,Name from singer order by Age desc"
1,medium gold,"SELECT name , country , age FROM singer ORDER BY age DESC"
2,medium pred,"select Song_Name,Name,Country from singer order by Age desc"
3,medium gold,"SELECT name , country , age FROM singer ORDER BY age DESC"
4,medium pred,"select avg(Age),min(Age),max(Age) from singer where Age = (select max(Age) from singer)"
...,...,...
1472,medium gold,"SELECT T2.feature_type_name FROM Other_Available_Features AS T1 JOIN Ref_Feature_Types AS T2 ON T1.feature_type_code = T2.feature_type_code WHERE T1.feature_name = ""AirCon"""
1473,medium pred,select property_type_description from Ref_Property_Types
1474,medium gold,SELECT T2.property_type_description FROM Properties AS T1 JOIN Ref_Property_Types AS T2 ON T1.property_type_code = T2.property_type_code GROUP BY T1.property_type_code
1475,hard pred,select property_name from Properties where agreed_selling_price > 'terminal'


## Select only the rows that has gold in the description of the query

In [378]:
gold_values = filtered_data[filtered_data['Column1'].str.contains('gold')]['Column2']
gold_values_df = pd.DataFrame(gold_values, columns=['Column2'])
gold_values_df

,Column2
1,"SELECT name , country , age FROM singer ORDER BY age DESC"
3,"SELECT name , country , age FROM singer ORDER BY age DESC"
5,"SELECT avg(age) , min(age) , max(age) FROM singer WHERE country = 'France'"
7,"SELECT song_name , song_release_year FROM singer ORDER BY age LIMIT 1"
9,SELECT song_name FROM singer WHERE age > (SELECT avg(age) FROM singer)
...,...
1468,SELECT Citizenship FROM singer WHERE Birth_Year < 1945 INTERSECT SELECT Citizenship FROM singer WHERE Birth_Year > 1955
1470,SELECT Citizenship FROM singer WHERE Birth_Year < 1945 INTERSECT SELECT Citizenship FROM singer WHERE Birth_Year > 1955
1472,"SELECT T2.feature_type_name FROM Other_Available_Features AS T1 JOIN Ref_Feature_Types AS T2 ON T1.feature_type_code = T2.feature_type_code WHERE T1.feature_name = ""AirCon"""
1474,SELECT T2.property_type_description FROM Properties AS T1 JOIN Ref_Property_Types AS T2 ON T1.property_type_code = T2.property_type_code GROUP BY T1.property_type_code


## Now the task is to force the comparison between sets of queries, here we lower every string and remove empty spaces

In [379]:
gold_values_df["Column2"] = gold_values_df["Column2"].str.strip().str.lower()
dev_gold["sql"] = dev_gold["sql"].str.strip().str.lower()
gold_values_df

,Column2
1,"select name , country , age from singer order by age desc"
3,"select name , country , age from singer order by age desc"
5,"select avg(age) , min(age) , max(age) from singer where country = 'france'"
7,"select song_name , song_release_year from singer order by age limit 1"
9,select song_name from singer where age > (select avg(age) from singer)
...,...
1468,select citizenship from singer where birth_year < 1945 intersect select citizenship from singer where birth_year > 1955
1470,select citizenship from singer where birth_year < 1945 intersect select citizenship from singer where birth_year > 1955
1472,"select t2.feature_type_name from other_available_features as t1 join ref_feature_types as t2 on t1.feature_type_code = t2.feature_type_code where t1.feature_name = ""aircon"""
1474,select t2.property_type_description from properties as t1 join ref_property_types as t2 on t1.property_type_code = t2.property_type_code group by t1.property_type_code


In [380]:
dev_gold

,database,question,sql
0,poker_player,How many poker players are there?,select count(*) from poker_player
1,poker_player,Count the number of poker players.,select count(*) from poker_player
2,poker_player,List the earnings of poker players in descending order.,select earnings from poker_player order by earnings desc
3,poker_player,"What are the earnings of poker players, ordered descending by value?",select earnings from poker_player order by earnings desc
4,poker_player,List the final tables made and the best finishes of poker players.,"select final_table_made , best_finish from poker_player"
...,...,...,...
115,singer,What are the names of the singers that have more than one songs?,select t1.name from singer as t1 join song as t2 on t1.singer_id = t2.singer_id group by t1.name having count(*) > 1
116,singer,Show the names of singers and the total sales of their songs.,"select t1.name , sum(t2.sales) from singer as t1 join song as t2 on t1.singer_id = t2.singer_id group by t1.name"
117,singer,"For each singer name, what is the total sales for their songs?","select t1.name , sum(t2.sales) from singer as t1 join song as t2 on t1.singer_id = t2.singer_id group by t1.name"
118,singer,List the name of singers that do not have any song.,select name from singer where singer_id not in (select singer_id from song)


In [381]:
gold_values_df["Column2"] = gold_values_df["Column2"].str.replace(' ', '')
dev_gold["sql"] = dev_gold["sql"].str.replace(' ', '')

In [382]:
gold_values_df

,Column2
1,"selectname,country,agefromsingerorderbyagedesc"
3,"selectname,country,agefromsingerorderbyagedesc"
5,"selectavg(age),min(age),max(age)fromsingerwherecountry='france'"
7,"selectsong_name,song_release_yearfromsingerorderbyagelimit1"
9,selectsong_namefromsingerwhereage>(selectavg(age)fromsinger)
...,...
1468,selectcitizenshipfromsingerwherebirth_year<1945intersectselectcitizenshipfromsingerwherebirth_year>1955
1470,selectcitizenshipfromsingerwherebirth_year<1945intersectselectcitizenshipfromsingerwherebirth_year>1955
1472,"selectt2.feature_type_namefromother_available_featuresast1joinref_feature_typesast2ont1.feature_type_code=t2.feature_type_codewheret1.feature_name=""aircon"""
1474,selectt2.property_type_descriptionfrompropertiesast1joinref_property_typesast2ont1.property_type_code=t2.property_type_codegroupbyt1.property_type_code


In [383]:
sql_gold= gold_values_df["Column2"].to_list()
sql_gold

['selectname,country,agefromsingerorderbyagedesc',
 'selectname,country,agefromsingerorderbyagedesc',
 "selectavg(age),min(age),max(age)fromsingerwherecountry='france'",
 'selectsong_name,song_release_yearfromsingerorderbyagelimit1',
 'selectsong_namefromsingerwhereage>(selectavg(age)fromsinger)',
 'selectsong_namefromsingerwhereage>(selectavg(age)fromsinger)',
 'selectlocation,namefromstadiumwherecapacitybetween5000and10000',
 'selectname,capacityfromstadiumorderbyaveragedesclimit1',
 'selectname,capacityfromstadiumorderbyaveragedesclimit1',
 'selectcount(*)fromconcertwhereyear=2014oryear=2015',
 'selectcount(*)fromconcertwhereyear=2014oryear=2015',
 'selectt2.name,count(*)fromconcertast1joinstadiumast2ont1.stadium_id=t2.stadium_idgroupbyt1.stadium_id',
 'selectt2.name,count(*)fromconcertast1joinstadiumast2ont1.stadium_id=t2.stadium_idgroupbyt1.stadium_id',
 'selectt2.name,t2.capacityfromconcertast1joinstadiumast2ont1.stadium_id=t2.stadium_idwheret1.year>=2014groupbyt2.stadium_idorder

## Calculate how many queries can be found from the development evaluation of Syntax Sql in our development set. 
### Reminder we are calculating the number of incorrect queries. Because the evaluation script of spider only shows the wrong queries

In [384]:
index_match = []

i = 0
for value in dev_gold['sql']:
    if value in sql_gold:
        index_match.append(i)
    i+=1

index_match

[2,
 3,
 4,
 5,
 10,
 11,
 12,
 13,
 16,
 17,
 20,
 21,
 22,
 23,
 36,
 37,
 41,
 42,
 44,
 46,
 48,
 49,
 50,
 51,
 52,
 56,
 57,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 82,
 83,
 86,
 87,
 88,
 89,
 90,
 91,
 98,
 99,
 100,
 101,
 102,
 103,
 108,
 109,
 114,
 115,
 116,
 117,
 118,
 119]

## Number of wrong queries using the predicitons of dev_result_eval Augmented

In [385]:
print(len(index_match))
# The number of wrong queries for augmented data is 63
# The number of wrong queries for non-augmented data is 69
print(f"Percentage of correct queries is {(len(dev_gold)-len(index_match))/len(dev_gold)*100}%")

63
Percentage of correct queries is 47.5%


## DO the analysis for all development of the spider dataset


In [41]:
data_pre_selected = pd.read_csv("/Users/yuriy/Desktop/Neural-Rule-Based-model-for-Text-to-SQL/data/devALL.csv", index_col=0)
data_pre_selected

,database,question,sql
0,concert_singer,How many singers do we have?,SELECT count(*) FROM singer
1,concert_singer,What is the total number of singers?,SELECT count(*) FROM singer
2,concert_singer,"Show name, country, age for all singers ordere...","SELECT name , country , age FROM singer ORDE..."
3,concert_singer,"What are the names, countries, and ages for ev...","SELECT name , country , age FROM singer ORDE..."
4,concert_singer,"What is the average, minimum, and maximum age ...","SELECT avg(age) , min(age) , max(age) FROM s..."
...,...,...,...
953,singer,List the name of singers that do not have any ...,SELECT Name FROM singer WHERE Singer_ID NOT IN...
954,singer,What is the sname of every sing that does not ...,SELECT Name FROM singer WHERE Singer_ID NOT IN...
955,real_estate_properties,How many available features are there in total?,SELECT count(*) FROM Other_Available_Features
956,real_estate_properties,What is the feature type name of feature AirCon?,SELECT T2.feature_type_name FROM Other_Availab...


In [65]:
data = pd.read_csv("/Users/yuriy/Downloads/generated_datasets/generated_data/saved_models/dev_result_eval.txt", sep="\t", header=None)
data

,0
0,"medium pred: select Song_Name,Name,Age from si..."
1,"medium gold: SELECT name , country , age FRO..."
2,"medium pred: select min(Age),avg(Age),max(Age)..."
3,"medium gold: SELECT avg(age) , min(age) , ma..."
4,"medium pred: select min(Age),avg(Age),max(Age)..."
...,...
1653,group 0.571 0.42...
1654,order 0.583 0.23...
1655,and/or 0.990 0.92...
1656,IUEN 1.000 1.00...


In [66]:
#filtered_data= data[:1477]# <---- For the augmented version SyntaxSQLAugmented
filtered_data= data[:1621] # <------- For the normal version of SyntaxSQL

In [67]:
dev_gold = pd.read_csv("/Users/yuriy/Desktop/spider/dev_gold.sql",sep="\t", header=None )
dev_gold

,0,1
0,SELECT count(*) FROM singer,concert_singer
1,SELECT count(*) FROM singer,concert_singer
2,"SELECT name , country , age FROM singer ORDE...",concert_singer
3,"SELECT name , country , age FROM singer ORDE...",concert_singer
4,"SELECT avg(age) , min(age) , max(age) FROM s...",concert_singer
...,...,...
1029,SELECT Citizenship FROM singer WHERE Birth_Yea...,singer
1030,SELECT count(*) FROM Other_Available_Features,real_estate_properties
1031,SELECT T2.feature_type_name FROM Other_Availab...,real_estate_properties
1032,SELECT T2.property_type_description FROM Prope...,real_estate_properties


In [68]:
dev_gold = dev_gold.rename(columns={0: "sql", 1: "database"})

In [69]:
filtered_data = filtered_data[0].str.split(":", n=1, expand=True)
filtered_data.columns = ['Column1', 'Column2']
filtered_data

,Column1,Column2
0,medium pred,"select Song_Name,Name,Age from singer order b..."
1,medium gold,"SELECT name , country , age FROM singer ORD..."
2,medium pred,"select min(Age),avg(Age),max(Age) from singer..."
3,medium gold,"SELECT avg(age) , min(age) , max(age) FROM ..."
4,medium pred,"select min(Age),avg(Age),max(Age) from singer"
...,...,...
1616,medium gold,SELECT T2.feature_type_name FROM Other_Availa...
1617,medium pred,select property_type_description from Ref_Pro...
1618,medium gold,SELECT T2.property_type_description FROM Prop...
1619,hard pred,select property_name from Properties where ro...


In [70]:
gold_values = filtered_data[filtered_data['Column1'].str.contains('gold')]['Column2']
gold_values_df = pd.DataFrame(gold_values, columns=['Column2'])
gold_values_df

,Column2
1,"SELECT name , country , age FROM singer ORD..."
3,"SELECT avg(age) , min(age) , max(age) FROM ..."
5,"SELECT avg(age) , min(age) , max(age) FROM ..."
7,"SELECT song_name , song_release_year FROM si..."
9,"SELECT song_name , song_release_year FROM si..."
...,...
1612,SELECT Citizenship FROM singer WHERE Birth_Ye...
1614,SELECT Citizenship FROM singer WHERE Birth_Ye...
1616,SELECT T2.feature_type_name FROM Other_Availa...
1618,SELECT T2.property_type_description FROM Prop...


In [71]:
gold_values_df["Column2"] = gold_values_df["Column2"].str.strip().str.lower()
dev_gold["sql"] = dev_gold["sql"].str.strip().str.lower()
gold_values_df

,Column2
1,"select name , country , age from singer orde..."
3,"select avg(age) , min(age) , max(age) from s..."
5,"select avg(age) , min(age) , max(age) from s..."
7,"select song_name , song_release_year from sin..."
9,"select song_name , song_release_year from sin..."
...,...
1612,select citizenship from singer where birth_yea...
1614,select citizenship from singer where birth_yea...
1616,select t2.feature_type_name from other_availab...
1618,select t2.property_type_description from prope...


In [72]:
gold_values_df["Column2"] = gold_values_df["Column2"].str.replace(' ', '')
dev_gold["sql"] = dev_gold["sql"].str.replace(' ', '')

In [73]:
sql_gold= gold_values_df["Column2"].to_list()
sql_gold

['selectname,country,agefromsingerorderbyagedesc',
 "selectavg(age),min(age),max(age)fromsingerwherecountry='france'",
 "selectavg(age),min(age),max(age)fromsingerwherecountry='france'",
 'selectsong_name,song_release_yearfromsingerorderbyagelimit1',
 'selectsong_name,song_release_yearfromsingerorderbyagelimit1',
 'selectdistinctcountryfromsingerwhereage>20',
 'selectcountry,count(*)fromsingergroupbycountry',
 'selectsong_namefromsingerwhereage>(selectavg(age)fromsinger)',
 'selectsong_namefromsingerwhereage>(selectavg(age)fromsinger)',
 'selectlocation,namefromstadiumwherecapacitybetween5000and10000',
 'selectname,capacityfromstadiumorderbyaveragedesclimit1',
 'selectname,capacityfromstadiumorderbyaveragedesclimit1',
 'selectcount(*)fromconcertwhereyear=2014oryear=2015',
 'selectcount(*)fromconcertwhereyear=2014oryear=2015',
 'selectt2.name,count(*)fromconcertast1joinstadiumast2ont1.stadium_id=t2.stadium_idgroupbyt1.stadium_id',
 'selectt2.name,count(*)fromconcertast1joinstadiumast2on

In [74]:
index_match = []

i = 0
for value in dev_gold['sql']:
    if value in sql_gold:
        index_match.append(i)
    i+=1

index_match

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 26,
 27,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 47,
 48,
 51,
 52,
 53,
 54,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 89,
 90,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 155,
 156,
 157,
 158,
 159,
 160,
 163,
 164,
 165,
 166,
 167,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 193,
 194,
 197,
 198,
 199,
 200,
 201,
 202,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 2

In [75]:
print(len(index_match))
# The number of wrong queries for augmented data is 63
# The number of wrong queries for non-augmented data is 69
print(f"Percentage of correct queries is {(len(dev_gold)-len(index_match))/len(dev_gold)*100}%")

836
Percentage of correct queries is 19.148936170212767%
